In [1]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import math
import glob
import re
import collections
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking,Dropout,Conv1D,TimeDistributed,Flatten,GRU,Bidirectional,MaxPooling1D
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.optimizers import Adam
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'ฆ': 99, 'ฝ': 122, 'f': 70, 'ฤ': 129, '3': 20, '=': 30, 'ั': 141, 'า': 142, '๓': 168, 'ภ': 125, 'ึ': 146, 'ุ': 148, 'x': 89, '&': 7, 'ว': 131, 'y': 90, 'ร': 128, '๒': 167, ']': 62, 'G': 40, 'B': 35, 'ข': 95, 'ฉ': 102, 'ห': 135, '6': 23, 'H': 41, 's': 84, 'โ': 153, 'N': 47, 'F': 39, "'": 8, '๔': 169, 'ฑ': 110, '+': 12, '1': 18, '}': 92, 'g': 71, 'ิ': 144, 'o': 79, 'ฟ': 124, 'V': 55, '๕': 170, 'เ': 151, 'j': 74, 'ฯ': 139, 't': 85, '\\': 61, '9': 26, 'ญ': 106, '/': 16, 'ก': 94, '็': 158, '๐': 165, '์': 163, '0': 17, 'W': 56, 'e': 69, '>': 31, 'other': 80, 'ใ': 154, 'Y': 58, '^': 63, 'c': 67, 'แ': 152, '‘': 175, 'v': 87, ')': 10, '’': 176, 'บ': 119, 'k': 75, '@': 33, 'u': 86, 'R': 51, ':': 27, 'ผ': 121, ' ': 1, '่': 159, '$': 5, 'จ': 101, 'ฏ': 108, 'd': 68, 'A': 34, '๖': 171, 'ซ': 104, 'K': 44, '"': 3, 'a': 65, 'ๅ': 156, 'X': 57, 'ธ': 117, 'ฬ': 136, 'ม': 126, '_': 64, 'อ': 137, '!': 2, 'ท': 116, '~': 93, 'ฐ': 109, 'ษ': 133, 'i': 73, 'น': 118, '7': 24, 'ต': 114, 'p': 81, 'z': 91, 'ู': 149,

In [3]:
###################################
### my model for tokenize words ###
###################################
# from keras.models import Model
# from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
# from keras.layers import Input, GRU, Bidirectional
# from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

##########################
####load model weight ####
##########################
# weight_path_my_best_model2='/data/model_best2.h5'
# my_best_model2 = get_my_best_model2()
# my_best_model2.load_weights(weight_path_my_best_model2)
# #my_best_model2.make_predict_function()
# my_best_model2.summary()

In [4]:
# Path to the preprocessed data
best_processed_path = '/data/Wongnai/w_review_train.csv'
test_path = '/data/Wongnai/test_file.csv'
#Changing .csv file to dataframe without header(0 = review, 1 = rating)
df = pd.read_csv(best_processed_path, sep=';', header=None)
df_test = pd.read_csv(test_path, sep=';', header=None)
# print("--------- TRAIN ---------")
# print(df[0][:10],df[1][:10])
# print("--------- TEST ---------")
# print(df_test[0][:10],df_test[1][:10])
print(df[1][:10])

0    3
1    4
2    3
3    5
4    5
5    4
6    4
7    3
8    5
9    5
Name: 1, dtype: int64


In [5]:
###################
## Cleaning data ##
###################

# ########################## New review data ##########################
# new_df = []
# for i,x in enumerate (df[0]):
#     new_df.append((re.sub(r'[$|.|!|~|=|*|\n|-|#|+|>|<|)|(|_|^|%|:D|?|,|-|"|-|฿|0|1|2|3|4|5|6|7|8|9|0|\t|-|;|:|●|&|@|/|-|{|}|[|]|”|“|]',r'',x)))

# ########################## New Test data ##########################
# new_test_df = []
# for i,x in enumerate (df_test[1]):
#     new_test_df.append((re.sub(r'[$|.|!|~|=|*|\n|-|#|+|>|<|)|(|_|^|%|:D|?|,|-|"|-|฿|0|1|2|3|4|5|6|7|8|9|0|\t|-|;|:|●|&|@|/|-|{|}|[|]|”|“|]',r'',x)))


In [6]:
#############################
# Create y_train label data #
#############################
label = []
for rate in df[1]:
   label.append(rate)
print(label[:10])
y_train = pd.get_dummies(pd.Series(label)).as_matrix()
print(y_train[:10])

[3, 4, 3, 5, 5, 4, 4, 3, 5, 5]
[[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [7]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [8]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    word_token = ("|".join(list(filter(lambda x: len(x.replace(" ", "")) > 0, word_token))))
    return word_token

In [9]:
#########################
# Cutkum New_cleaing data
#########################
# new_data2 =[]
# for i in tqdm(range(len(new_df))):
#     new_data2.append(cutKumKrub(new_df[i]))
# print(new_data2[1])
# #Train Data
#Test Data

# test_data2 =[]
# for i in tqdm(range(len(new_test_df))):
#     test_data2.append(cutKumKrub(new_test_df[i]))
# print(test_data2[1])
# #Test Data
# with open('/data/Wongnai/test_review2.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(test_data2, f, pickle.HIGHEST_PROTOCOL)

In [10]:
###################################
# Loading data_review from disk   #
###################################
def load_data(path_data):
#     data_review = []
    with open(path_data, 'rb') as f:
        data_review = pickle.load(f)
    return data_review

data_review = []
train_data_path = '/data/Wongnai/data_review2.pickle'
data_review = load_data(train_data_path)
print(data_review[39999])

###########################
# Change string to list   #
###########################
def senToWord(data_review):
    temp = []
    data_review2 = []
    for sentence_index in tqdm(range(len(data_review))):
        for y in data_review[sentence_index].split('|'):
            #temp.append(y.strip('\n'))
            temp.append(re.sub(r'[$|.|!|~|\n|-|#|+|>|<]',r'',y))
        data_review2.append(temp)
        temp = []
    return data_review2

# data_review2 = []
data_review = senToWord(data_review)
print(data_review[39999])
print(len(data_review[39999]))
#print(data_review2[39997][399])
#print(data_review2[39997][-1])

100%|██████████| 40000/40000 [00:05<00:00, 6857.17it/s]

วัน|นี้|มา|กิน|กัน|ไกล|ถึง|แม่|สอด|จังหวัด|ตากติด|กับ|ชาย|แดน|กัน|ไป|เลย|อำเภอ|นี้|เพราะ|ฉะนั้น|มา|ถึง|ที่|นี่|แล้ว|ก็|ต้อง|กิน|อาหาร|ท้องถิ่น|กัน|หน่อย|อาหาร|ท้องถิ่น|ที่|นี่|มี|หลากหลาย|โดย|เฉพาะ|อาหาร|ที่|มา|จาก|ฝั่ง|เพื่อนบ้าน|ชื่อ|อ่าน|ยาก|มาก|ๆ|ครับ|ดู|ตาม|เมนู|ที่|ผม|โพสต์|ไว้|ได้|เลย|แต่|เมนูอร่อย|ครับ|แปลก|ใหม่|มาก|ๆ|ด้วย|ไม่|เคย|กิน|ที่|ไหน|มา|ก่อน|เป็น|ประสบการณ์|ใหม่|อีก|ร้าน|เลย|ครับ|ร้าน|นี้|มา|แล้ว|ลอง|แวะ|เวียนหา|ประสบการณ์|ใหม่|ใหม่|กัน|นะ|ครับ|อร่อย|ดี|แปลก|ใหม่|ไม่|เหมือน|ใคร|โดย|เฉพาะ|เมนู|แลพัด|โตะ
['วัน', 'นี้', 'มา', 'กิน', 'กัน', 'ไกล', 'ถึง', 'แม่', 'สอด', 'จังหวัด', 'ตากติด', 'กับ', 'ชาย', 'แดน', 'กัน', 'ไป', 'เลย', 'อำเภอ', 'นี้', 'เพราะ', 'ฉะนั้น', 'มา', 'ถึง', 'ที่', 'นี่', 'แล้ว', 'ก็', 'ต้อง', 'กิน', 'อาหาร', 'ท้องถิ่น', 'กัน', 'หน่อย', 'อาหาร', 'ท้องถิ่น', 'ที่', 'นี่', 'มี', 'หลากหลาย', 'โดย', 'เฉพาะ', 'อาหาร', 'ที่', 'มา', 'จาก', 'ฝั่ง', 'เพื่อนบ้าน', 'ชื่อ', 'อ่าน', 'ยาก', 'มาก', 'ๆ', 'ครับ', 'ดู', 'ตาม', 'เมนู', 'ที่', 'ผม', 'โพสต์', 'ไว้', 'ได้', 'เ

In [11]:
###############################
## load test Data for get score
###############################
#Load test path
test_review = []
test_data_path = '/data/Wongnai/test_review2.pickle'
test_review = load_data(test_data_path)
print(test_review[1])

# test_review2 = []
test_review = senToWord(test_review)
test_review = test_review[1:]
print(test_review[0])
print(len(test_review[0]))

100%|██████████| 6204/6204 [00:00<00:00, 6604.71it/s]

ร้าน|นี้|จะ|อยู่|เส้น|สัน|กำแพง|-|แม่ออน|เลย|แยก|บ่อสร้าง|ร้าน|จะ|อยู่|ด้าน|ซ้าย|ติด|ริม|ถนน|มี|ป้าย|ติด|ไว้|เห็น|ชัดเจน|ปูทอง|ข้าว|แกง|รส|เด็ด|ตาม|หา|ข้าว|แกง|รสชาติ|นี้|มา|ตลอด|ใน|ที่สุด|ก็|ได้|เจอ|เพราะ|ส่วน|ใหญ่|จะ|เจอรสชาติ|กลาง|ๆ|ถ้า|คน|ชอบรส|จัดจ้าน|แต่|ไม่|ถึง|กับ|เผ็ดเว่อร์|แนะ|นำ|ร้าน|นี้|เลย|ค่ะ|ที่|นั่ง|ก็|สะอาด|สะอ้าน|มี|ทั้ง|น้ำ|ซุป|และ|น้ำพริก|กะปิฟรี|และ|น้ำ|ดื่ม|ฟรีบริการ|ตัว|เอง|คหสต|ชอบ|แบบ|นี้|ที่|ไม่|บังคับ|ให้|ลูกค้า|ต้อง|ซื้อ|น้ำ|กิน|เพราะ|ทาน|คน|เดียว|แค่|ขวด|เหลือ|ทิ้ง|ตลอด|เสียดายเงิน|สั่ง|กับ|ข้าว|ราด|มา|อย่าง|อร่อย|ทั้ง|สอง|อย่าง|เสียดาย|ไข่|ดาว|หมด|ซะ|ก่อน|แต่|แค่|อย่าง|กับ|ข้าว|สวย|ที่|ให้|มา|ใน|ปริมาณ|ที่|อิ่ม|พอดี|ก็|เต็มที่|แล้ว|แม่|ค้า|ก็|อัธยาศัย|ดี|รับรอง|ว่า|มี|กลับ|มา|กิน|อีก|บ่อย|ๆ|แน่นอน|ใคร|ผ่าน|มา|แถว|นี้|ลอง|แวะ|ชิม|นะ|คะ|สำหรับ|คน|ที่|ชอบรสกลม|กล่อม|ไม่|จืด|ชืด
['ร้าน', 'นี้', 'จะ', 'อยู่', 'เส้น', 'สัน', 'กำแพง', '-', 'แม่ออน', 'เลย', 'แยก', 'บ่อสร้าง', 'ร้าน', 'จะ', 'อยู่', 'ด้าน', 'ซ้าย', 'ติด', 'ริม', 'ถนน', 'มี', 'ป้าย', 'ติด', 'ไว้', 'เ

In [12]:
##################
# Make embedding # Now use thai2vec
##################
# from tqdm import tqdm
# def file_len(fname):
#     with open(fname) as f:
#         for i, l in enumerate(f):
#             pass
#     return i + 1

# word2vec = dict()
# fileLen = file_len('/data/thai2vec.vec')
# with open('/data/thai2vec.vec', 'r') as f:
#     #f.seek(0)
#     for index, line in tqdm(enumerate(f), total = fileLen):
#         if index == 0:
#             continue
#         word2vec[line.split(' ')[0]] = [float(x) for x in line.split(' ')[1:]]
# print(len(word2vec))
# with open('/data/Wongnai/embedding2.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(word2vec, f, pickle.HIGHEST_PROTOCOL)

In [13]:
words =[]
for sentence in data_review:
    for word in sentence:
        words.append(word)
word_count = list()
word_count.extend(collections.Counter(words).most_common(len(set(words))))
print("[Total unique word] =",len(word_count))
threshold = 3

num_UNK = 0
index = len(word_count) - 1
print(index)
rare_word = set()
while(word_count[index][1] <= threshold):
    num_UNK += word_count[index][1]
    rare_word.add(word_count[index][0])
    index -= 1
print("[UNK word] =",len(rare_word)*100/len(word_count)) 

word_count = word_count[:index+1]
word_count.append(("UNK",num_UNK))
word_count = sorted(word_count, key=lambda x: -x[1])

#print out 10 most frequent words
print(word_count[-10:])
dictionary = dict()
dictionary["for_keras_zero_padding"] = 0
for word in word_count:
    dictionary[word[0]] = len(dictionary)
data = list()
data_feature = []
for x in data_review:
    for word in x:
        if(word not in rare_word):
            data.append(dictionary[word])
        else:
            data.append(dictionary["UNK"])
    data_feature.append(data)
    data =[]
# print(list(map(lambda lis: list(map(lambda y: index_to_word[y], lis)), data_feature[:2])))
print("[Unique word] =",len(word_count))
del words 

[Total unique word] = 129802
129801
[UNK word] = 82.19056717153819
[('เบค่อนกรอบ', 4), ('บางขุนศรี', 4), ('Tsui', 4), ('Nama', 4), ('นไป', 4), ('เค้กตัด', 4), ('update', 4), ('frappucino', 4), ('สเต็คปลา', 4), ('เฉียดร้อย', 4)]
[Unique word] = 23118


In [14]:
###############################
# Loading embedding Once time #
###############################
embedding_path = '/data/Wongnai/embedding2.pickle'
pre_em = load_data(embedding_path)
print(len(pre_em))

51358


In [15]:
#word to index, index to key form fastText emb
word_to_index = dict()
index_to_word = dict()
for idx, x in enumerate(pre_em.keys()):
    word_to_index[x] = idx
    index_to_word[idx] = x

In [16]:
#Check different words between DataSet and Embed
print(len(dictionary)) # Dataset
print(len(word_to_index)) # 
count = 0
notmatch_word =[]
for i in dictionary.keys():
    if(i in word_to_index):
        count+=1
    else:
        notmatch_word.append(i)
print("match word =",count,"percent =",count*100/len(dictionary))

23119
51358
match word = 7479 percent = 32.35001513906311


In [17]:
# Create embedding Main on review train_data
pre_embedding = []
index =[]
pre_embedding.append(np.zeros(300))
for i,l in enumerate (dictionary.keys()):
    if(l in word_to_index):
        index.append(l)
        pre_embedding.append(pre_em[l])
    else:
        pre_embedding.append(np.zeros(300))
weight_em = np.array(pre_embedding)
print(weight_em.shape)

(23120, 300)


In [18]:
# #Create embedding Main train Embeding ### This mean Original Embedding + unknow_words Union
# pre_embedding_ftext = []
# pre_embedding_ftext.append(np.zeros(300))
# for i,sen in enumerate (pre_em):
#     pre_embedding_ftext.append(pre_em[sen])
# # print(pre_embedding_ftext[1])
# #-------------------------------------------
# print(len(dictionary)) # Dataset
# print(len(word_to_index)) # 
# count = 0
# notmatch_word =[]
# for i in dictionary.keys():
#     if(i in word_to_index.keys()):
#         count+=1
#     else:
#         notmatch_word.append(i)
# print("match word =",count,"percent =",count*100 /len(dictionary))
# #--------------------------------------------
# word_to_index2 = []
# word_to_index2 = word_to_index
# for i,l in enumerate (dictionary.keys()):
#     if(not l in word_to_index2):
#         word_to_index2[l] = len(word_to_index2)
#         pre_embedding_ftext.append(np.zeros(300)) 
# #---------------------------------------------
# print(len(dictionary)) # Dataset
# print(len(word_to_index2)) # 
# count = 0
# notmatch_word =[]
# for i in dictionary.keys():
#     if(i in word_to_index2.keys()):
#         count+=1
#     else:
#         notmatch_word.append(i)
# print("match word =",count,"percent =",count*100 /len(dictionary))

In [19]:
# word_to_index2['UNK']

In [20]:
# #prepare feature now use thai2vec
# data_feature_ftext= []
# data =[]
# for sentence in data_review:
#     for word in sentence:
#         if(word in word_to_index2):
#             data.append(word_to_index2[word])
#         else:
#             data.append(word_to_index2["UNK"])
#     data_feature_ftext.append(data)
#     data =[]

In [21]:
# print(len(pre_embedding_ftext),pre_embedding_ftext[130469])

In [22]:
# weight_em_ftext = np.array(pre_embedding_ftext)
# con = []
# count=0
# weight_em_ftext_new  =[]
# for i,word in enumerate (weight_em_ftext):
#     if (len(word) != 300):
#         weight_em_ftext_new.append(np.zeros(300))
#     else:
#         weight_em_ftext_new.append(weight_em_ftext[i])

# weight_em_ftext_new = np.array(weight_em_ftext_new)
# print(count, con, weight_em_ftext_new.shape)


In [23]:
#Make padding for train data OLDDDD  ###################################################################################
def pad(A, npads):
    _npads = npads - len(A)
    return np.pad(A, pad_width=_npads, mode='constant', constant_values=0)[_npads:]

x_train = sequence.pad_sequences(data_feature,maxlen =1000, padding= 'post', truncating='pre')

print(x_train.shape)

(40000, 1000)


In [24]:
#Make input feature for test data OLDDDDDDDDD ###########################################################################
data =[]
test_feature = []
for x in test_review:
    for word in x:
        if(word in dictionary.keys()):
            data.append(dictionary[word])
        else:
            data.append(dictionary["UNK"])
    test_feature.append(data)
    data =[]
test =[]

test = sequence.pad_sequences(test_feature,maxlen =1000, padding= 'post', truncating='pre')
print(test.shape)

(6203, 1000)


In [25]:
# #Make padding for train data NEWWWWWWWWWWW
# x_train = np.array(data_feature_ftext)
# x_train2 = []
# for i in range (len(x_train)):
#     x_train2.append(pad(x_train[i],400))
# x_train = np.array(x_train2)
# print(x_train.shape)

In [26]:
# #Make input feature for test data NEW
# data_test_ftext= []
# data =[]
# for sentence in test_review:
#     for word in sentence:
#         if(word in word_to_index2):
#             data.append(word_to_index2[word])
#         else:
#             data.append(word_to_index2["UNK"])
#     data_test_ftext.append(data)
#     data =[]
# test =[]
# for i in range (len(data_test_ftext)):
#     test.append(pad(x_train[i],400))
# test = np.array(test)
# print(test.shape)

In [27]:
##Prepare data for training
input_data = x_train
train_data = input_data[:int(len(input_data)*0.7)]
val_data = input_data[int(len(input_data)*0.7):int(len(input_data)*0.9)]
test_data_ev = input_data[int(len(input_data)*0.9):]

target = y_train
train_target = y_train[:int(len(target)*0.7)]
val_target = y_train[int(len(target)*0.7):int(len(target)*0.9)]
test_target_ev = label[int(len(target)*0.9):]

In [28]:
len(train_data)

28000

In [29]:
from sklearn.metrics import f1_score,precision_score,recall_score
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)
    correct = 0
    #map probability to class
    y_pred_mapped = []
    for i,pred in enumerate(y_pred):
        pred = list(pred)
        y_pred_mapped.append(pred.index(max(pred))+1)

#             y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

    f1score = f1_score(y_test,y_pred_mapped, average='weighted')
    precision = precision_score(y_test,y_pred_mapped, average='weighted')
    recall = recall_score(y_test,y_pred_mapped, average='weighted')
    return f1score, precision, recall

In [30]:
# def get_my_best_model_br():
#     input1 = Input(shape=(1000,))
#     x = Embedding(len(dictionary)+1,300, weights = [weight_em])(input1)
#     x = Conv1D(64,5,strides=1,activation='relu',padding="valid")(x)
#     x = MaxPooling1D(pool_size=5, strides=1, padding='valid')(x)
#     x = Conv1D(32,5,strides=1,activation='relu',padding="valid")(x)
#     x = MaxPooling1D(pool_size=5, strides=1, padding='valid')(x)
# #     x = TimeDistributed(Dense(10))(x)
# #     x = Bidirectional(GRU(50))(x)
#     x = Dropout(0.25)(x)
#     x = TimeDistributed(Dense(5))(x)
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dense(100, activation='relu')(x)
# #     x = Dropout(0.4)(x)
#     out = Dense(5, activation='softmax')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='categorical_crossentropy',
#                  metrics=['categorical_accuracy'])
#     return model
# # #load model weight
# # weight_path_my_best_model2='/data/model_best2.h5'
# model = get_my_best_model_br()
# # my_best_model2.load_weights(weight_path_my_best_model2)
# #model.make_predict_function()
# model.summary()

In [31]:
# %time
# weight_path_model='/data/Wongnai/my_weight_CNN.h5'
# callbacks_list_model = [
#     ModelCheckpoint(
#         weight_path_model,
#         monitor = "val_loss",
#         mode = 'min',
#         verbose = 1,
#         save_best_only = True,
#         save_weights_only = True,
#     )   
# ]
# model.fit(train_data,train_target,batch_size=256,epochs=5,verbose=1,callbacks=callbacks_list_model,
#           validation_data=(val_data, val_target))

In [32]:
# # #load model weight
# weight_path_my_best_model='/data/Wongnai/my_weight_CNN.h5'
# model = get_my_best_model_br()
# model.load_weights(weight_path_my_best_model)
# # #my_best_model2.make_predict_function()
# # model.summary()

In [33]:
# # test_target_ev = np.argmax(test_target_ev, axis=1)
# evaluate(test_data_ev,test_target_ev,model)
# y_pred=model.predict(test) 
# print(y_pred[0])

In [34]:
# ans = np.argmax(y_pred, axis=1)
# ans = [x+1 for x in ans]
# d =dict()
# d['reviewID'] = [x for x in range (1,6204)]
# d['rating'] = ans
# sub= pd.DataFrame(d)
# # sub.to_csv('/data/BestCnn.csv', ',', index=False , columns=['reviewID','rating'])
# sub.to_csv('/data/BestCnn.csv', ',', index=False , columns=['reviewID','rating'])

In [35]:
#####################################################################################################

In [41]:
def get_model():
    input1 = Input(shape=(1000,))
    x = Embedding(len(dictionary)+1,300, weights = [weight_em])(input1)
    x = Conv1D(32,5,strides=1,activation='relu',padding="valid")(x)
    x = Conv1D(32,5,strides=1,activation='relu',padding="valid")(x)
    x = MaxPooling1D(pool_size=5, strides=1, padding='valid')(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(75, activation='relu')(x)
    x = Dense(75, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(5, activation='softmax')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])
    return model
model = get_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1000, 300)         6936000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 996, 32)           48032     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 992, 32)           5152      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 988, 32)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 988, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 31616)             0         
__________

In [42]:
%time
weight_path_model='/data/Wongnai/my_weight_test.h5'
callbacks_list_model = [
   # TensorBoard(log_dir='/data/Graph/gru', histogram_freq=1, write_graph=True, write_grads=True),
    ModelCheckpoint(
        weight_path_model,
        monitor = "val_loss",
        mode = 'min',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    )   
]

# model.fit(x_train_char,y_train,batch_size=4096, epochs=3, verbose=2,
#              callbacks=callbacks_list_model, validation_data=(x_val_char,y_val))
model.fit(train_data,train_target,batch_size=128,epochs=5,verbose=1,callbacks=callbacks_list_model,
          validation_data=(val_data, val_target))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.68 µs
Train on 28000 samples, validate on 8000 samples
Epoch 1/5
28000/28000 [==============================] - 19s - loss: 1.2354 - categorical_accuracy: 0.4560 - val_loss: 1.1611 - val_categorical_accuracy: 0.4796
Epoch 2/5
28000/28000 [==============================] - 18s - loss: 1.0723 - categorical_accuracy: 0.5203 - val_loss: 0.9979 - val_categorical_accuracy: 0.5455
Epoch 3/5
28000/28000 [==============================] - 18s - loss: 0.8864 - categorical_accuracy: 0.6063 - val_loss: 0.9954 - val_categorical_accuracy: 0.5496
Epoch 4/5
28000/28000 [==============================] - 18s - loss: 0.6533 - categorical_accuracy: 0.7279 - val_loss: 1.1619 - val_categorical_accuracy: 0.5179
Epoch 5/5
28000/28000 [==============================] - 18s - loss: 0.4076 - categorical_accuracy: 0.8446 - val_loss: 1.4288 - val_categorical_accuracy: 0.5145


In [43]:
# #load model weight
weight_path_my_best_model='/data/Wongnai/my_weight_test.h5'
model = get_model()
model.load_weights(weight_path_my_best_model)
# #my_best_model2.make_predict_function()
# model.summary()

In [39]:
# evaluate(test_data_ev,test_target_ev,model)

/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.48847782918082611, 0.52004143782873735, 0.53600000000000003)

In [44]:
y_pred=model.predict(test) 
print(y_pred[0])
ans = np.argmax(y_pred, axis=1)
ans = [x+1 for x in ans]
d = dict()
d['reviewID'] = [x for x in range (1,6204)]
d['rating'] = ans
sub= pd.DataFrame(d)
sub.to_csv('/data/Cnn1.csv', ',', index=False , columns=['reviewID','rating'])

[  4.42610281e-06   6.64945284e-04   1.18724301e-01   5.64647496e-01
   3.15958798e-01]
